In [2]:
import pandas as pd
from sodapy import Socrata
from api_keys import my_app_token
from api_keys import chicago_password
from api_keys import chicago_username






In [7]:

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
#client = Socrata("data.cityofchicago.org", None)

#Example authenticated client (needed for non-public datasets):
client = Socrata("data.cityofchicago.org",
                my_app_token,
                username=chicago_username",
                password=chicago_password)

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("m6dm-c72p", limit=2000)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

results_df

,trip_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,fare,...,additional_charges,trip_total,shared_trip_authorized,trips_pooled,pickup_centroid_latitude,pickup_centroid_longitude,pickup_centroid_location,dropoff_centroid_latitude,dropoff_centroid_longitude,dropoff_centroid_location
0,a8faf6bfd5a4269a6e629937b96d27898297ccae,2022-12-15T19:00:00.000,2022-12-15T19:15:00.000,399,1.24,17031839100,17031833100,32,28,17.5,...,1.24,18.74,False,1,41.8809944707,-87.6327464887,"{'type': 'Point', 'coordinates': [-87.63274648...",41.8790669938,-87.657005027,"{'type': 'Point', 'coordinates': [-87.65700502..."
1,a8faf716448c1bd3803e77d66e803f6d0776a17f,2022-12-05T20:45:00.000,2022-12-05T20:45:00.000,420,1.9741,17031320400,17031081300,32,8,5,...,4.67,9.67,False,1,41.8774061234,-87.6219716519,"{'type': 'Point', 'coordinates': [-87.62197165...",41.8983317935,-87.6207628651,"{'type': 'Point', 'coordinates': [-87.62076286..."
2,a8faf8c0c0b7aef6b09d7a9b3006a3f2ef9a3e57,2022-12-04T13:15:00.000,2022-12-04T13:15:00.000,389,1.65,17031838100,17031833000,28,28,10,...,1.24,11.24,False,1,41.8786674201,-87.6716536214,"{'type': 'Point', 'coordinates': [-87.67165362...",41.8852813201,-87.6572331997,"{'type': 'Point', 'coordinates': [-87.65723319..."
3,a8faf9360a36e12bdc55b2a536618901e1550f87,2022-12-08T07:30:00.000,2022-12-08T07:45:00.000,1105,3.8895,NaN,NaN,3,7,17.5,...,3.39,20.89,False,1,41.96581197,-87.6558787862,"{'type': 'Point', 'coordinates': [-87.65587878...",41.9226862843,-87.6494887289,"{'type': 'Point', 'coordinates': [-87.64948872..."
4,a8fafa1f7090f98947356d78c803d2ea13d5f7c1,2022-12-05T20:15:00.000,2022-12-05T20:15:00.000,407,2.3052,17031833000,17031081500,28,8,7.5,...,4.74,12.24,False,1,41.8852813201,-87.6572331997,"{'type': 'Point', 'coordinates': [-87.65723319...",41.8925077809,-87.6262149064,"{'type': 'Point', 'coordinates': [-87.62621490..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,a90fa0eeb007a6f6aa9ad47c65c020774c41e885,2022-12-03T09:45:00.000,2022-12-03T10:00:00.000,653,2.0849,17031081300,17031081800,8,8,7.5,...,2.91,10.41,False,1,41.8983317935,-87.6207628651,"{'type': 'Point', 'coordinates': [-87.62076286...",41.8932163595,-87.6378442095,"{'type': 'Point', 'coordinates': [-87.63784420..."
1996,a90fa6faa066eeb330d7abfcc9a2c85b4df6bf21,2022-12-16T14:00:00.000,2022-12-16T14:15:00.000,607,2.1119,17031140302,17031160601,14,16,7.5,...,2.97,11.47,False,1,41.9725977673,-87.7237819658,"{'type': 'Point', 'coordinates': [-87.72378196...",41.9595267906,-87.7020985409,"{'type': 'Point', 'coordinates': [-87.70209854..."
1997,a90faa7d375afcd239bdb1be0f3c9b4db4442f2e,2022-12-03T21:30:00.000,2022-12-03T21:45:00.000,428,0.9525,17031081403,17031081700,8,8,7.5,...,3.42,10.92,False,1,41.8909220259,-87.6188683546,"{'type': 'Point', 'coordinates': [-87.61886835...",41.8920421365,-87.6318639497,"{'type': 'Point', 'coordinates': [-87.63186394..."
1998,a90faa9e1536e72aa30dbc37a74a579437e09b5b,2022-12-14T21:00:00.000,2022-12-14T21:30:00.000,1909,10.45,17031833100,17031030101,28,77,42.5,...,151.24,223.74,False,1,41.8790669938,-87.657005027,"{'type': 'Point', 'coordinates': [-87.65700502...",41.9912345323,-87.6573042698,"{'type': 'Point', 'coordinates': [-87.65730426..."
